# chroma_luma
This notebook explores the various chroma (color) and luma (brightness) features that can be extracted from individual frames. We'll be making extensive use of the `OpenCV` computer vision library. Functionality demonstrated here will be replicated in a functions file, and then used to populate the frame-level DataFrame.

In [1]:
import cv2
import os

In [2]:
film = 'parasite'
frame = 933
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'

### Loading an image
We can load the image and get its width and height.

In [3]:
# this frame is 854 (width) x 358 (height) pixels and has a BGR value
image = cv2.imread(img_path)
image.shape

(358, 854, 3)

We can specify a (width, height) coordinate and get the BGR value, consisting of three color channels.

In [4]:
image[10][20]

array([53, 74, 71], dtype=uint8)

# Blank Frames
The easiest frame to spot is the blank frame, with either all black or all white pixels. These could be used to identify scene transitions, such as when a scene ends with a dip-to-black or dip-to-white, transitioning to blank frames before the next scene.

### All-black frames
We can calculate the mean luminosity of the image by calculating the average of every BGR value for every pixel.

In [5]:
film = 'parasite'
frame = 22
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'
image = cv2.imread(img_path)

In [6]:
# every pixel in this frame has a BGR value of (0, 0, 0)
image.mean()

0.0

In [7]:
if image.mean() < 3: # threshold of 3, to be safe
    print('black frame detected')

black frame detected


### All-white frames

In [8]:
if image.mean() > 252:
    print('white frame detected')

# Luma
Measurements relating to luma.
### Mean brightness
We can take the brightness of the frame by calculating the average brightness of each pixel. But because each pixel is a color BGR element, we must first convert the image to grayscale, because brightness doesn't map linearly to BGR values. After converting each pixels' three BGR values to a single grayscale value, we can take the mean of the pixels' grayscale values.

In [9]:
film = 'booksmart'
frame = 4102
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'
image = cv2.imread(img_path)

In [10]:
# this is NOT the proper way to calculate brightness
image.mean()

114.32336262926611

In [11]:
# this conversion to grayscale is necessary
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray.mean()

111.82475067757808

### Maximum brightness
Although black values always sit at the end of the spectrum (0, 0, 0), white values in H.264-compressed videos rarely approach the pure white point of (255, 255, 255) because of gamma correction, a bandwidth-saving technique. On a display, the human eye doesn't need to see pure white to interpret pure white. Brightness is a power-scale, not a linear-scale, and so gamma correction can be used to reduce the amount of data used to convey white.

Below, we can search for the frame's brightest pixel. We may use this to scale white values.

In [12]:
brightest_mean = 0
brightest_pixel = 0
brightest_coordinate = 0
x = 0


for a in image:
    y = 0
    for b in a:
        if b.mean() > brightest_mean:
            brightest_mean = b.mean()
            brightest_pixel = b
            brightest_coordinate = (x, y)
        y += 1
    x += 1
print(brightest_mean)
print(brightest_pixel)
print(brightest_coordinate)

208.33333333333334
[221 211 193]
(4, 412)
